In [1]:
# 协同过滤算法UCF：给用户推荐和他兴趣相似的其他用户喜欢的产品

In [2]:
# 余弦相似度
from sklearn.metrics.pairwise import cosine_similarity

i = [1, 0, 0, 0]
j = [1, 0, 1, 1]
k = [0, 1, 1, 0]

cosine_similarity([i, j, k]) # 生成一个3*3的矩阵，表示i,j,k之间的相似度

array([[1.        , 0.57735027, 0.        ],
       [0.57735027, 1.        , 0.40824829],
       [0.        , 0.40824829, 1.        ]])

In [3]:
# 皮尔逊相关系数
from scipy.stats import pearsonr

i = [1, 0, 0, 0]
j = [1, 0, 1, 1]

pearsonr(i, j) # 返回两个向量的相关系数和p值

PearsonRResult(statistic=0.3333333333333332, pvalue=0.6666666666666667)

In [6]:
import numpy as np
import pandas as pd

# 加载数据，交互记录以字典的形式存储，因为交互矩阵太稀疏
def loadData():
    users = {'Alice': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
             'user1': {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
             'user2': {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
             'user3': {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
             'user4': {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
             }
    return users

user_data = loadData()
user_data

{'Alice': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
 'user1': {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
 'user2': {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
 'user3': {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
 'user4': {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}}

In [7]:
# 计算用户相似度矩阵
similarity_matrix = pd.DataFrame(
    np.identity(len(user_data)),
    index=user_data.keys(),
    columns=user_data.keys(),
) # 初始化相似度矩阵 对角线上的值为1

print(f'init similarity_matrix: \n{similarity_matrix}\n')

for u1, items1 in user_data.items(): 
    for u2, items2 in user_data.items():
        if u1 == u2:
            continue
        vec1, vec2 = [], []
        # items1是u1的交互记录，items2是u2的交互记录
        for item, rating1 in items1.items():
            rating2 = items2.get(item, -1) # 如果u2没有交互过这个item，返回-1
            if rating2 == -1:
                continue
            vec1.append(rating1)
            vec2.append(rating2)
        # 计算不同用户之间的皮尔逊相关系数
        similarity_matrix[u1][u2] = np.corrcoef(vec1, vec2)[0][1]

print(f'similarity_matrix: \n{similarity_matrix}\n')

init similarity_matrix: 
       Alice  user1  user2  user3  user4
Alice    1.0    0.0    0.0    0.0    0.0
user1    0.0    1.0    0.0    0.0    0.0
user2    0.0    0.0    1.0    0.0    0.0
user3    0.0    0.0    0.0    1.0    0.0
user4    0.0    0.0    0.0    0.0    1.0

similarity_matrix: 
          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000



In [8]:
# 计算用户Alice的knn
target_user = 'Alice'
n = 2
sim_users = similarity_matrix[target_user].sort_values(ascending=False)[1:n+1].index.tolist()
print(f'与用户{target_user}最相似的{n}个用户为：{sim_users}\n')

与用户Alice最相似的2个用户为：['user1', 'user2']



In [ ]:
# 预测用户Alice对物品E的评分
weighted_scores = 0.
corr_values_sum = 0.

target_item = 'E'
# 基于皮尔逊相关系数预测用户评分
for user in sim_users:
    corr_value = similarity_matrix[target_user][user]
    user_mean_rating = np.mean(list(user_data[user].values()))

    weighted_scores += corr_value * (user_data[user][target_item] - user_mean_rating)
    corr_values_sum += corr_value

target_user_mean_rating = np.mean(list(user_data[target_user].values()))
target_item_pred = target_user_mean_rating + weighted_scores / corr_values_sum
print(f'用户{target_user}对物品{target_item}的预测评分为：{target_item_pred}')


用户Alice对物品E的预测评分为：4.871979899370592


In [5]:
# 评估指标
# 召回率Recall：模型推荐的物品中用户实际喜欢的物品 / 用户实际喜欢的物品总数
# 精确率Precision：模型推荐的物品中用户实际喜欢的物品 / 模型推荐的物品总数
# 覆盖率Coverage：模型推荐的物品数 / 物品总数